# Step 7 — Position-Aware Final Output

Turn scored candidates into the final 8–10 items for the UI:

- **Position 1:** anchor head score or D-t-D override (engagement hook)
- **Positions 2–10:** remaining by full business score
- **Explanation:** one-line for position 1 from top contributing feature
- **Cap:** max 200 candidates enter ranking → output top 8–10

**What Step 7 is doing**

- **Input:** Scored candidates per session (business_score + anchor_score from Step 5).
- **1. Cap:** Only the top 200 candidates by business score are kept; the rest are dropped to keep the pipeline fast and bounded.
- **2. Step 6:** The capped list goes through post-ranking (diversity, category mix, price shock, margin cap, time-of-day, D-t-D). If a distance-to-discount nudge applies, Step 6 already puts that item at position 1 with its message.
- **3. Position 1:** If there was no D-t-D override, we set position 1 to the item with the **highest anchor_score** (the model that predicts "accepted when shown at position 1"), so the best "hook" is always in slot 1.
- **4. Positions 2–10:** The remaining slots stay ordered by **full business score** (blend of accept, AOV, abandon, timing, anchor).
- **5. Explanation:** The position-1 item gets a one-line explanation from the top contributing feature (e.g. beverage/dessert/bread gap, complement, bestseller, or the D-t-D nudge text).

**Files used:** Existing data and models only (no new files in `data/` or `models/`). Step 7 logic lives in `position_aware_output.py`; `__pycache__` is created automatically when that module is imported.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from position_aware_output import position_aware_final_output, MAX_CANDIDATES_FOR_RANKING, DEFAULT_RAIL_SIZE

DATA_DIR = os.path.abspath("../data")
MODEL_DIR = os.path.abspath("../models")

# Step 7 — Position-Aware Final Output

Turn scored candidates into the final 8–10 items for the UI:

- **Position 1:** anchor head score or D-t-D override (engagement hook)
- **Positions 2–10:** remaining by full business score
- **Explanation:** one-line for position 1 from top contributing feature
- **Cap:** max 200 candidates enter ranking → output top 8–10

Step 7 logic is in `position_aware_output.py` (see imports above). It caps candidates at 200, runs Step 6 post-ranking, then sets position 1 to D-t-D override or best anchor; positions 2–10 by business score; one-line explanation for position 1.

In [ ]:
# Run Step 7 on sample sessions
np.random.seed(42)
sids = np.random.choice(test_df["session_id"].unique(), size=min(100, test_df["session_id"].nunique()), replace=False)
results = []
for sid in sids:
    sess = test_df[test_df["session_id"] == sid].copy()
    if len(sess) < 3:
        continue
    ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
    pm = sess["peak_hour_mode"].iloc[0]
    rail, st = position_aware_final_output(sess, ct, cs, pm, rail_size=DEFAULT_RAIL_SIZE)
    results.append({"session_id": sid, **st})

stats_df = pd.DataFrame(results)
print(f"Sessions: {len(stats_df)}")
print(f"Input → after_cap → output: {stats_df['input'].mean():.0f} → {stats_df['after_cap'].mean():.0f} → {stats_df['output'].iloc[0]}")
print(f"Position 1 from D-t-D override: {(stats_df['dtd_override']==1).sum()} sessions")

**Before vs after Step 7 — position 1 and explanation**

For one session: (A) top 10 by business score only vs (B) after Step 7 (position 1 = anchor or D-t-D, with explanation).

**What Step 7 is doing**

- **Input:** Scored candidates per session (business_score + anchor_score from Step 5).
- **1. Cap:** Only the top 200 candidates by business score are kept; the rest are dropped to keep the pipeline fast and bounded.
- **2. Step 6:** The capped list goes through post-ranking (diversity, category mix, price shock, margin cap, time-of-day, D-t-D). If a distance-to-discount nudge applies, Step 6 already puts that item at position 1 with its message.
- **3. Position 1:** If there was no D-t-D override, we set position 1 to the item with the **highest anchor_score** (the model that predicts "accepted when shown at position 1"), so the best "hook" is always in slot 1.
- **4. Positions 2–10:** The remaining slots stay ordered by **full business score** (blend of accept, AOV, abandon, timing, anchor).
- **5. Explanation:** The position-1 item gets a one-line explanation from the top contributing feature (e.g. beverage/dessert/bread gap, complement, bestseller, or the D-t-D nudge text).

**Files used:** Existing data and models only (no new files in `data/` or `models/`). Step 7 logic lives in `position_aware_output.py`; `__pycache__` is created automatically when that module is imported.

**Position 1 source** — % of sessions where position 1 came from D-t-D override vs from anchor score.

In [ ]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from position_aware_output import position_aware_final_output, MAX_CANDIDATES_FOR_RANKING, DEFAULT_RAIL_SIZE

DATA_DIR = os.path.abspath("../data")
MODEL_DIR = os.path.abspath("../models")

**Explanation types** — Distribution of position-1 explanation text (from top contributing feature or D-t-D nudge).

In [ ]:
# Collect position-1 explanation for each session in sample
explanation_counts = []
for sid in stats_df["session_id"]:
    sess = test_df[test_df["session_id"] == sid].copy()
    if len(sess) < 3:
        continue
    ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
    rail, _ = position_aware_final_output(sess, ct, cs, sess["peak_hour_mode"].iloc[0])
    expl = rail.iloc[0].get("explanation", "") or ""
    explanation_counts.append(expl)
expl_series = pd.Series(explanation_counts).value_counts()
print("Position-1 explanation distribution:")
display(expl_series.head(10))

**Cap impact** — Max 200 candidates enter ranking; output is top 8–10. Below: one session where the cap applies (if any), else summary.

In [ ]:
# Sessions with most candidates (cap applies when input > 200)
cap_example = stats_df.loc[stats_df["input"].idxmax()]
print(f"Session with most candidates: {cap_example['input']} → after cap: {cap_example['after_cap']} → output: {cap_example['output']}")
if cap_example["input"] > MAX_CANDIDATES_FOR_RANKING:
    print(f"Cap applied: {cap_example['input']} candidates trimmed to {MAX_CANDIDATES_FOR_RANKING} before Step 6.")
else:
    print(f"No session in sample had >{MAX_CANDIDATES_FOR_RANKING} candidates; cap applies when candidate pool is larger.")
print(f"\nAcross sample: input mean = {stats_df['input'].mean():.0f}, after_cap mean = {stats_df['after_cap'].mean():.0f}, output = {stats_df['output'].iloc[0]} items")

**Step 7 flow** — Scored candidates → Cap 200 → Step 6 post-rank → Position 1 (D-t-D or anchor) → Pos 2–10 by biz score → Rail 8–10 + explanation.

In [ ]:
# Example: one session final rail
sid = sids[0]
sess = test_df[test_df["session_id"] == sid].copy()
ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
rail, _ = position_aware_final_output(sess, ct, cs, sess["peak_hour_mode"].iloc[0])
display(rail[["rank", "item_name", "item_category", "item_price", "business_score", "explanation"]])

**Step 7 done.** Position 1 = anchor or D-t-D; 2–10 by business score; explanation from top feature; max 200 candidates → top 8–10. Logic in `position_aware_output.py`; uses existing data and models (no new files in `data/` or `models/`).

In [ ]:
# Load data, models, prepare features, score test set
from lgbm_ranker import (
    engineer_labels, prepare_features, temporal_split,
    compute_business_score, load_models,
)
features = pd.read_csv(f"{DATA_DIR}/training_features.csv")
sessions = pd.read_csv(f"{DATA_DIR}/sessions.csv")
menu = pd.read_csv(f"{DATA_DIR}/menu_items.csv")
gru_hidden = np.load(f"{DATA_DIR}/gru_hidden_states.npy")

features = engineer_labels(features)
features, feature_cols, encoders = prepare_features(features, gru_hidden)
_, _, test_mask = temporal_split(features, sessions)
models = load_models()

test_df = features[test_mask].copy()
preds = {n: m.predict(test_df[feature_cols]) for n, m in models.items()}
test_df["business_score"] = compute_business_score(preds)
test_df["anchor_score"] = preds["anchor"]

for col in ["item_category", "item_subcategory", "peak_hour_mode"]:
    if col in encoders:
        test_df[col] = encoders[col].inverse_transform(test_df[col].astype(int))
test_df["item_name"] = test_df["item_id"].map(menu.set_index("item_id")["name"])

print(f"Scored test set: {len(test_df):,} rows")

Step 7 logic is in `position_aware_output.py` (see imports above). It caps candidates at 200, runs Step 6 post-ranking, then sets position 1 to D-t-D override or best anchor; positions 2–10 by business score; one-line explanation for position 1.

In [ ]:
# Run Step 7 on sample sessions
np.random.seed(42)
sids = np.random.choice(test_df["session_id"].unique(), size=min(100, test_df["session_id"].nunique()), replace=False)
results = []
for sid in sids:
    sess = test_df[test_df["session_id"] == sid].copy()
    if len(sess) < 3:
        continue
    ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
    pm = sess["peak_hour_mode"].iloc[0]
    rail, st = position_aware_final_output(sess, ct, cs, pm, rail_size=DEFAULT_RAIL_SIZE)
    results.append({"session_id": sid, **st})

stats_df = pd.DataFrame(results)
print(f"Sessions: {len(stats_df)}")
print(f"Input → after_cap → output: {stats_df['input'].mean():.0f} → {stats_df['after_cap'].mean():.0f} → {stats_df['output'].iloc[0]}")
print(f"Position 1 from D-t-D override: {(stats_df['dtd_override']==1).sum()} sessions")

**Before vs after Step 7 — position 1 and explanation**

For one session: (A) top 10 by business score only vs (B) after Step 7 (position 1 = anchor or D-t-D, with explanation).

# Step 7 — Position-Aware Final Output

Turn scored candidates into the final 8–10 items for the UI:
- **Position 1:** anchor head score or D-t-D override (engagement hook)
- **Positions 2–10:** remaining by full business score
- **Explanation:** one-line for position 1 from top contributing feature
- **Cap:** max 200 candidates enter ranking → output top 8–10

**Position 1 source** — % of sessions where position 1 came from D-t-D override vs from anchor score.

In [ ]:
dtd_pct = 100 * (stats_df["dtd_override"] == 1).mean()
anchor_pct = 100 - dtd_pct
print(f"Anchor (position-1 score): {anchor_pct:.1f}%  |  D-t-D override: {dtd_pct:.1f}%")

**What Step 7 is doing**

- **Input:** Scored candidates per session (business_score + anchor_score from Step 5).
- **1. Cap:** Only the top 200 candidates by business score are kept; the rest are dropped to keep the pipeline fast and bounded.
- **2. Step 6:** The capped list goes through post-ranking (diversity, category mix, price shock, margin cap, time-of-day, D-t-D). If a distance-to-discount nudge applies, Step 6 already puts that item at position 1 with its message.
- **3. Position 1:** If there was no D-t-D override, we set position 1 to the item with the **highest anchor_score** (the model that predicts “accepted when shown at position 1”), so the best “hook” is always in slot 1.
- **4. Positions 2–10:** The remaining slots stay ordered by **full business score** (blend of accept, AOV, abandon, timing, anchor).
- **5. Explanation:** The position-1 item gets a one-line explanation from the top contributing feature (e.g. beverage/dessert/bread gap, complement, bestseller, or the D-t-D nudge text).

**Files used:** Existing data and models only (no new files in `data/` or `models/`). Step 7 logic lives in `position_aware_output.py`; `__pycache__` is created automatically when that module is imported.

In [ ]:
# Collect position-1 explanation for each session in sample
explanation_counts = []
for sid in stats_df["session_id"]:
    sess = test_df[test_df["session_id"] == sid].copy()
    if len(sess) < 3:
        continue
    ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
    rail, _ = position_aware_final_output(sess, ct, cs, sess["peak_hour_mode"].iloc[0])
    expl = rail.iloc[0].get("explanation", "") or ""
    explanation_counts.append(expl)
expl_series = pd.Series(explanation_counts).value_counts()
print("Position-1 explanation distribution:")
display(expl_series.head(10))

**Cap impact** — Max 200 candidates enter ranking; output is top 8–10. Below: one session where the cap applies (if any), else summary.

In [21]:
import sys, os
sys.path.insert(0, os.path.abspath(".."))
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

from position_aware_output import position_aware_final_output, MAX_CANDIDATES_FOR_RANKING, DEFAULT_RAIL_SIZE

DATA_DIR = os.path.abspath("../data")
MODEL_DIR = os.path.abspath("../models")

**Step 7 flow** — Scored candidates → Cap 200 → Step 6 post-rank → Position 1 (D-t-D or anchor) → Pos 2–10 by biz score → Rail 8–10 + explanation.

In [ ]:
# Example: one session final rail
sid = sids[0]
sess = test_df[test_df["session_id"] == sid].copy()
ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
rail, _ = position_aware_final_output(sess, ct, cs, sess["peak_hour_mode"].iloc[0])
display(rail[["rank", "item_name", "item_category", "item_price", "business_score", "explanation"]])

**Step 7 done.** Position 1 = anchor or D-t-D; 2–10 by business score; explanation from top feature; max 200 candidates → top 8–10. Logic in `position_aware_output.py`; uses existing data and models (no new files in `data/` or `models/`).

In [22]:
# Load data, models, prepare features, score test set
from lgbm_ranker import (
    engineer_labels, prepare_features, temporal_split,
    compute_business_score, load_models,
)
features = pd.read_csv(f"{DATA_DIR}/training_features.csv")
sessions = pd.read_csv(f"{DATA_DIR}/sessions.csv")
menu = pd.read_csv(f"{DATA_DIR}/menu_items.csv")
gru_hidden = np.load(f"{DATA_DIR}/gru_hidden_states.npy")

features = engineer_labels(features)
features, feature_cols, encoders = prepare_features(features, gru_hidden)
_, _, test_mask = temporal_split(features, sessions)
models = load_models()

test_df = features[test_mask].copy()
preds = {n: m.predict(test_df[feature_cols]) for n, m in models.items()}
test_df["business_score"] = compute_business_score(preds)
test_df["anchor_score"] = preds["anchor"]

for col in ["item_category", "item_subcategory", "peak_hour_mode"]:
    if col in encoders:
        test_df[col] = encoders[col].inverse_transform(test_df[col].astype(int))
test_df["item_name"] = test_df["item_id"].map(menu.set_index("item_id")["name"])

print(f"Scored test set: {len(test_df):,} rows")

OSError: dlopen(/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib
  Referenced from: <D44045CD-B874-3A27-9A61-F131D99AACE4> /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/lightgbm/lib/lib_lightgbm.dylib
  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/local/lib/libomp/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/local/lib/libomp/libomp.dylib' (no such file)

In [9]:
# Step 7 logic is in position_aware_output.py (see imports above)

In [23]:
# Run Step 7 on sample sessions
np.random.seed(42)
sids = np.random.choice(test_df["session_id"].unique(), size=min(100, test_df["session_id"].nunique()), replace=False)
results = []
for sid in sids:
    sess = test_df[test_df["session_id"] == sid].copy()
    if len(sess) < 3:
        continue
    ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
    pm = sess["peak_hour_mode"].iloc[0]
    rail, st = position_aware_final_output(sess, ct, cs, pm, rail_size=DEFAULT_RAIL_SIZE)
    results.append({"session_id": sid, **st})

stats_df = pd.DataFrame(results)
print(f"Sessions: {len(stats_df)}")
print(f"Input → after_cap → output: {stats_df['input'].mean():.0f} → {stats_df['after_cap'].mean():.0f} → {stats_df['output'].iloc[0]}")
print(f"Position 1 from D-t-D override: {(stats_df['dtd_override']==1).sum()} sessions")

Sessions: 50
Input → after_cap → output: 13 → 13 → 5
Position 1 from D-t-D override: 0 sessions


**Before vs after Step 7 — position 1 and explanation**

For one session: (A) top 10 by business score only vs (B) after Step 7 (position 1 = anchor or D-t-D, with explanation).

In [24]:
# Same session: before Step 7 (top 10 by business score) vs after Step 7 (position 1 = anchor or D-t-D + explanation)
sid = sids[0]
sess = test_df[test_df["session_id"] == sid].copy()
ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
pm = sess["peak_hour_mode"].iloc[0]

before = sess.sort_values("business_score", ascending=False).head(10).reset_index(drop=True)
before["rank"] = range(1, len(before) + 1)
rail_after, st = position_aware_final_output(sess, ct, cs, pm)

print("Before Step 7 — top 10 by business score only (position 1 = highest business_score):")
display(before[["rank", "item_name", "item_category", "business_score"]].head(5))
print("\nAfter Step 7 — position 1 = anchor or D-t-D; one-line explanation for slot 1:")
display(rail_after[["rank", "item_name", "item_category", "business_score", "explanation"]].head(5))
print(f"\nPosition 1 changed: {before.iloc[0]['item_name']} → {rail_after.iloc[0]['item_name']}")
print(f"Explanation for position 1: \"{rail_after.iloc[0]['explanation']}\"")

Before Step 7 — top 10 by business score only (position 1 = highest business_score):


,rank,item_name,item_category,business_score
0,1,Kesari Bath,main,0.333094
1,2,Mint Raita,beverage,0.330736
2,3,Laccha Paratha,appetizer,0.322490
3,4,Roasted Papad,beverage,0.207021
4,5,Rajma Masala,beverage,0.112070



After Step 7 — position 1 = anchor or D-t-D; one-line explanation for slot 1:


,rank,item_name,item_category,business_score,explanation
0,1,Mint Raita,beverage,0.330736,Recommended for you!
1,2,Kesari Bath,main,0.333094,Recommended for you!
2,3,Laccha Paratha,appetizer,0.322490,
3,4,Roasted Papad,beverage,0.207021,
4,5,Rajma Masala,beverage,0.112070,



Position 1 changed: Kesari Bath → Mint Raita
Explanation for position 1: "Recommended for you!"


**Position 1 source** — % of sessions where position 1 came from D-t-D override vs from anchor score.

In [ ]:
dtd_pct = 100 * (stats_df["dtd_override"] == 1).mean()
anchor_pct = 100 - dtd_pct
print(f"Anchor (position-1 score): {anchor_pct:.1f}%  |  D-t-D override: {dtd_pct:.1f}%")

**Explanation types** — Distribution of position-1 explanation text (from top contributing feature or D-t-D nudge).

In [ ]:
# Collect position-1 explanation for each session in sample
explanation_counts = []
for sid in stats_df["session_id"]:
    sess = test_df[test_df["session_id"] == sid].copy()
    if len(sess) < 3:
        continue
    ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
    rail, _ = position_aware_final_output(sess, ct, cs, sess["peak_hour_mode"].iloc[0])
    expl = rail.iloc[0].get("explanation", "") or ""
    explanation_counts.append(expl)
expl_series = pd.Series(explanation_counts).value_counts()
print("Position-1 explanation distribution:")
display(expl_series.head(10))

**Cap impact** — Max 200 candidates enter ranking; output is top 8–10. Below: one session where the cap applies (if any), else summary.

In [ ]:
# Sessions with most candidates (cap applies when input > 200)
cap_example = stats_df.loc[stats_df["input"].idxmax()]
print(f"Session with most candidates: {cap_example['input']} → after cap: {cap_example['after_cap']} → output: {cap_example['output']}")
if cap_example["input"] > MAX_CANDIDATES_FOR_RANKING:
    print(f"Cap applied: {cap_example['input']} candidates trimmed to {MAX_CANDIDATES_FOR_RANKING} before Step 6.")
else:
    print(f"No session in sample had >{MAX_CANDIDATES_FOR_RANKING} candidates; cap applies when candidate pool is larger.")
print(f"\nAcross sample: input mean = {stats_df['input'].mean():.0f}, after_cap mean = {stats_df['after_cap'].mean():.0f}, output = {stats_df['output'].iloc[0]} items")

**Step 7 flow** — Scored candidates → Cap 200 → Step 6 post-rank → Position 1 (D-t-D or anchor) → Pos 2–10 by biz score → Rail 8–10 + explanation.

In [ ]:
# Step 7 pipeline (text summary; see markdown above for flow)
pass

In [ ]:
# Example: one session final rail
sid = sids[0]
sess = test_df[test_df["session_id"] == sid].copy()
ct, cs = sess["cart_total_value"].iloc[0], int(sess["cart_size"].iloc[0])
rail, _ = position_aware_final_output(sess, ct, cs, sess["peak_hour_mode"].iloc[0])
display(rail[["rank", "item_name", "item_category", "item_price", "business_score", "explanation"]])

**Step 7 done.** Position 1 = anchor or D-t-D; 2–10 by business score; explanation from top feature; max 200 candidates → top 8–10. Logic in `position_aware_output.py`; uses existing data and models (no new files in `data/` or `models/`).